# Graphs

*Modeling relationships and connections between entities*


## 🎯 Learning Objectives

- Understand graph terminology (vertices, edges, directed/undirected, weighted/unweighted, connected components)
- Implement graph representations: adjacency list and adjacency matrix, with trade-off analysis
- Perform BFS and DFS traversals and understand their applications (shortest path, connectivity, cycle detection)
- Implement topological sort for DAGs and understand its applications in dependency resolution
- Apply graph patterns: multi-source BFS, bipartite checking, Union-Find for dynamic connectivity

---
## 1. Introduction to Graphs


We've studied linear structures (arrays, linked lists) and hierarchical structures (trees). Now we explore **graphs**—the most general data structure for representing relationships. Trees are actually a special case of graphs!

> 📖 **Definition:** A **graph** G = (V, E) consists of a set of **vertices** (or nodes) V and a set of **edges** E connecting pairs of vertices. Unlike trees, graphs can have cycles, multiple paths between nodes, and nodes with any number of connections.

> 💡 **Graph vs Tree**
>
> ```

Tree (special graph):           General Graph:

        A                           A --- B
       /|\                          |\ /| |
      B C D                         | X | |
     /|                             |/ \| |
    E F                             C --- D

- One root, one path             - No root, multiple paths
- n-1 edges for n nodes          - Any number of edges
- No cycles                      - Can have cycles (A-B-D-C-A)
                
```

### Real-World Graph Examples

| Domain | Vertices | Edges |
| --- | --- | --- |
| Social Network | Users | Friendships |
| Road Map | Intersections | Roads |
| Web | Pages | Hyperlinks |
| Course Prerequisites | Courses | Dependencies |

**Listing 9.1 — Graph Basics**

In [ ]:
# Simple graph representation
# Vertices: A, B, C, D
# Edges: A-B, A-C, B-C, B-D, C-D

# As adjacency list (most common)
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B', 'D'],
    'D': ['B', 'C']
}

print("Graph (adjacency list):")
for vertex, neighbors in graph.items():
    print(f"  {vertex}: {neighbors}")

# Basic properties
num_vertices = len(graph)
num_edges = sum(len(neighbors) for neighbors in graph.values()) // 2
print(f"\nVertices: {num_vertices}")
print(f"Edges: {num_edges}")

# Check if edge exists - O(degree)
def has_edge(graph, u, v):
    return v in graph.get(u, [])

print(f"\nEdge A-B exists: {has_edge(graph, 'A', 'B')}")
print(f"Edge A-D exists: {has_edge(graph, 'A', 'D')}")

***Figure 9.1:** Adjacency list is the most common graph representation for sparse graphs.*

---
## 2. Graph Terminology


> 📖 **Key Terms:** **Directed vs Undirected:** In a directed graph (digraph), edges have direction (A→B ≠ B→A). In undirected graphs, edges are bidirectional.

**Weighted:** Edges may have weights (costs, distances, capacities).

**Degree:** Number of edges connected to a vertex. In digraphs: in-degree and out-degree.

**Path:** Sequence of vertices connected by edges.

**Cycle:** Path that starts and ends at the same vertex.

**Connected:** There's a path between every pair of vertices.

**Listing 9.2 — Directed vs Undirected Graphs**

In [ ]:
# Undirected graph - edges go both ways
undirected = {
    'A': ['B', 'C'],
    'B': ['A', 'C'],
    'C': ['A', 'B']
}

# Directed graph - edges have direction
directed = {
    'A': ['B', 'C'],  # A -> B, A -> C
    'B': ['C'],       # B -> C
    'C': []           # C has no outgoing edges
}

print("Undirected graph:")
for v, neighbors in undirected.items():
    print(f"  {v} <-> {neighbors}")

print("\nDirected graph:")
for v, neighbors in directed.items():
    print(f"  {v} -> {neighbors}")

# Degree calculations
def undirected_degree(graph, vertex):
    return len(graph[vertex])

def in_degree(graph, vertex):
    """Count incoming edges in directed graph."""
    count = 0
    for v in graph:
        if vertex in graph[v]:
            count += 1
    return count

def out_degree(graph, vertex):
    """Count outgoing edges in directed graph."""
    return len(graph[vertex])

print(f"\nUndirected degree of A: {undirected_degree(undirected, 'A')}")
print(f"Directed in-degree of C: {in_degree(directed, 'C')}")
print(f"Directed out-degree of A: {out_degree(directed, 'A')}")

***Figure 9.2:** Directed graphs distinguish between incoming and outgoing edges.*

**Listing 9.3 — Weighted Graphs**

In [ ]:
# Weighted graph - edges have costs
# Adjacency list with weights: {vertex: [(neighbor, weight), ...]}
weighted_graph = {
    'A': [('B', 4), ('C', 2)],
    'B': [('A', 4), ('C', 1), ('D', 5)],
    'C': [('A', 2), ('B', 1), ('D', 8)],
    'D': [('B', 5), ('C', 8)]
}

print("Weighted graph (city distances):")
for vertex, edges in weighted_graph.items():
    for neighbor, weight in edges:
        print(f"  {vertex} --{weight}--> {neighbor}")

# Alternative: dict of dicts
weighted_dict = {
    'A': {'B': 4, 'C': 2},
    'B': {'A': 4, 'C': 1, 'D': 5},
    'C': {'A': 2, 'B': 1, 'D': 8},
    'D': {'B': 5, 'C': 8}
}

print(f"\nWeight of edge A-B: {weighted_dict['A']['B']}")
print(f"Neighbors of C: {list(weighted_dict['C'].keys())}")

***Figure 9.3:** Weighted graphs store edge weights, useful for distances, costs, or capacities.*

---
## 3. Graph Representations


### Adjacency List

**Listing 9.4 — Adjacency List Implementation**

In [ ]:
from collections import defaultdict

class Graph:
    """Graph using adjacency list."""
    
    def __init__(self, directed=False):
        self.adj = defaultdict(list)
        self.directed = directed
    
    def add_vertex(self, v):
        """Ensure vertex exists."""
        if v not in self.adj:
            self.adj[v] = []
    
    def add_edge(self, u, v, weight=None):
        """Add edge between u and v."""
        if weight is not None:
            self.adj[u].append((v, weight))
            if not self.directed:
                self.adj[v].append((u, weight))
        else:
            self.adj[u].append(v)
            if not self.directed:
                self.adj[v].append(u)
    
    def neighbors(self, v):
        """Get neighbors of vertex."""
        return self.adj[v]
    
    def vertices(self):
        """Get all vertices."""
        return list(self.adj.keys())
    
    def __repr__(self):
        return f"Graph({dict(self.adj)})"

# Test
g = Graph(directed=False)
edges = [('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')]
for u, v in edges:
    g.add_edge(u, v)

print("Graph:")
for v in g.vertices():
    print(f"  {v}: {g.neighbors(v)}")

***Figure 9.4:** Adjacency list uses O(V + E) space and O(degree) time for neighbor lookup.*

### Adjacency Matrix

**Listing 9.5 — Adjacency Matrix Implementation**

In [ ]:
class GraphMatrix:
    """Graph using adjacency matrix."""
    
    def __init__(self, vertices):
        self.vertices = vertices
        self.v_index = {v: i for i, v in enumerate(vertices)}
        n = len(vertices)
        self.matrix = [[0] * n for _ in range(n)]
    
    def add_edge(self, u, v, weight=1):
        i, j = self.v_index[u], self.v_index[v]
        self.matrix[i][j] = weight
        self.matrix[j][i] = weight  # Undirected
    
    def has_edge(self, u, v):
        i, j = self.v_index[u], self.v_index[v]
        return self.matrix[i][j] != 0
    
    def neighbors(self, v):
        i = self.v_index[v]
        result = []
        for j, vertex in enumerate(self.vertices):
            if self.matrix[i][j] != 0:
                result.append(vertex)
        return result
    
    def display(self):
        print("    " + " ".join(self.vertices))
        for i, v in enumerate(self.vertices):
            print(f"{v}   " + " ".join(str(x) for x in self.matrix[i]))

# Test
g = GraphMatrix(['A', 'B', 'C', 'D'])
g.add_edge('A', 'B')
g.add_edge('A', 'C')
g.add_edge('B', 'C')
g.add_edge('B', 'D')
g.add_edge('C', 'D')

print("Adjacency Matrix:")
g.display()
print(f"\nNeighbors of B: {g.neighbors('B')}")
print(f"Edge A-D exists: {g.has_edge('A', 'D')}")

***Figure 9.5:** Adjacency matrix uses O(V²) space but provides O(1) edge lookup.*

### Comparison

| Operation | Adjacency List | Adjacency Matrix |
| --- | --- | --- |
| Space | O(V + E) | O(V²) |
| Add edge | O(1) | O(1) |
| Check edge | O(degree) | O(1) |
| Get neighbors | O(degree) | O(V) |
| Best for | Sparse graphs | Dense graphs |

**Listing 9.6 — Edge List Representation**

In [ ]:
# Edge list - simple but less efficient for queries
edges = [
    ('A', 'B', 4),
    ('A', 'C', 2),
    ('B', 'C', 1),
    ('B', 'D', 5),
    ('C', 'D', 8)
]

print("Edge list:")
for u, v, w in edges:
    print(f"  {u} --{w}--> {v}")

# Convert edge list to adjacency list
def edges_to_adj_list(edges, directed=False):
    adj = defaultdict(list)
    for u, v, w in edges:
        adj[u].append((v, w))
        if not directed:
            adj[v].append((u, w))
    return dict(adj)

adj = edges_to_adj_list(edges)
print(f"\nAs adjacency list: {adj}")

# Convert adjacency list to edge list
def adj_list_to_edges(adj, directed=False):
    edges = []
    seen = set()
    for u in adj:
        for v, w in adj[u]:
            edge = (min(u, v), max(u, v), w) if not directed else (u, v, w)
            if edge not in seen:
                edges.append((u, v, w))
                if not directed:
                    seen.add(edge)
    return edges

from collections import defaultdict

***Figure 9.6:** Edge list is simple to store but O(E) for most queries. Good for algorithms like Kruskal's MST.*

---
## 4. Breadth-First Search (BFS)


> 📖 **Definition:** **Breadth-First Search** explores a graph level by level, visiting all neighbors of a vertex before moving deeper. It uses a queue and finds the shortest path (in terms of edges) in unweighted graphs.

**Listing 9.7 — BFS Implementation**

In [ ]:
from collections import deque

def bfs(graph, start):
    """
    Breadth-First Search traversal.
    Time: O(V + E), Space: O(V)
    """
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = []
    
    while queue:
        vertex = queue.popleft()
        order.append(vertex)
        
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return order

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph:")
for v, neighbors in graph.items():
    print(f"  {v}: {neighbors}")

print(f"\nBFS from A: {bfs(graph, 'A')}")
print("Level by level: A -> B,C -> D,E,F")

***Figure 9.7:** BFS visits vertices level by level using a queue. First discovered = first visited.*

**Listing 9.8 — BFS Shortest Path**

In [ ]:
from collections import deque

def bfs_shortest_path(graph, start, end):
    """
    Find shortest path using BFS.
    Returns path and distance.
    """
    if start == end:
        return [start], 0
    
    visited = {start}
    queue = deque([(start, [start])])  # (vertex, path)
    
    while queue:
        vertex, path = queue.popleft()
        
        for neighbor in graph[vertex]:
            if neighbor == end:
                return path + [neighbor], len(path)
            
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    
    return None, -1  # No path found

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

path, dist = bfs_shortest_path(graph, 'A', 'F')
print(f"Shortest path A to F: {path}")
print(f"Distance: {dist} edges")

path, dist = bfs_shortest_path(graph, 'D', 'F')
print(f"\nShortest path D to F: {path}")
print(f"Distance: {dist} edges")

***Figure 9.8:** BFS naturally finds shortest path in unweighted graphs since it explores in order of distance.*

**Listing 9.9 — BFS Level Order**

In [ ]:
from collections import deque

def bfs_levels(graph, start):
    """BFS returning vertices grouped by level/distance."""
    visited = {start}
    queue = deque([start])
    levels = []
    
    while queue:
        level_size = len(queue)
        current_level = []
        
        for _ in range(level_size):
            vertex = queue.popleft()
            current_level.append(vertex)
            
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
        
        levels.append(current_level)
    
    return levels

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

levels = bfs_levels(graph, 'A')
print("BFS levels from A:")
for i, level in enumerate(levels):
    print(f"  Distance {i}: {level}")

***Figure 9.9:** BFS can group vertices by their distance from the start vertex.*

---
## 5. Depth-First Search (DFS)


> 📖 **Definition:** **Depth-First Search** explores as deep as possible along each branch before backtracking. It uses a stack (or recursion) and is useful for detecting cycles, finding paths, and exploring all possibilities.

**Listing 9.10 — DFS Recursive**

In [ ]:
def dfs_recursive(graph, start, visited=None):
    """
    Recursive DFS traversal.
    Time: O(V + E), Space: O(V) for recursion stack
    """
    if visited is None:
        visited = set()
    
    visited.add(start)
    order = [start]
    
    for neighbor in graph[start]:
        if neighbor not in visited:
            order.extend(dfs_recursive(graph, neighbor, visited))
    
    return order

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("DFS (recursive) from A:")
print(dfs_recursive(graph, 'A'))
print("\nGoes deep: A -> B -> D -> E -> F -> C")

***Figure 9.10:** Recursive DFS naturally follows depth-first exploration using the call stack.*

**Listing 9.11 — DFS Iterative**

In [ ]:
def dfs_iterative(graph, start):
    """
    Iterative DFS using explicit stack.
    Avoids recursion limit for large graphs.
    """
    visited = set()
    stack = [start]
    order = []
    
    while stack:
        vertex = stack.pop()
        
        if vertex in visited:
            continue
        
        visited.add(vertex)
        order.append(vertex)
        
        # Add neighbors in reverse for same order as recursive
        for neighbor in reversed(graph[vertex]):
            if neighbor not in visited:
                stack.append(neighbor)
    
    return order

# Compare recursive vs iterative
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print(f"DFS recursive:  {dfs_recursive(graph, 'A')}")
print(f"DFS iterative:  {dfs_iterative(graph, 'A')}")

***Figure 9.11:** Iterative DFS uses explicit stack, avoiding recursion limit issues.*

**Listing 9.12 — DFS Path Finding**

In [ ]:
def dfs_find_path(graph, start, end, path=None):
    """Find any path from start to end using DFS."""
    if path is None:
        path = []
    
    path = path + [start]
    
    if start == end:
        return path
    
    for neighbor in graph[start]:
        if neighbor not in path:  # Avoid cycles
            result = dfs_find_path(graph, neighbor, end, path)
            if result:
                return result
    
    return None  # No path found

def dfs_all_paths(graph, start, end, path=None):
    """Find ALL paths from start to end."""
    if path is None:
        path = []
    
    path = path + [start]
    
    if start == end:
        return [path]
    
    paths = []
    for neighbor in graph[start]:
        if neighbor not in path:
            new_paths = dfs_all_paths(graph, neighbor, end, path)
            paths.extend(new_paths)
    
    return paths

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print(f"One path A to F: {dfs_find_path(graph, 'A', 'F')}")
print(f"\nAll paths A to F:")
for path in dfs_all_paths(graph, 'A', 'F'):
    print(f"  {path}")

***Figure 9.12:** DFS can find one path or all paths between two vertices.*

---
## 6. Connected Components


**Listing 9.13 — Finding Connected Components**

In [ ]:
def find_connected_components(graph):
    """
    Find all connected components in undirected graph.
    Returns list of components (each a set of vertices).
    """
    visited = set()
    components = []
    
    def dfs(vertex, component):
        visited.add(vertex)
        component.add(vertex)
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                dfs(neighbor, component)
    
    for vertex in graph:
        if vertex not in visited:
            component = set()
            dfs(vertex, component)
            components.append(component)
    
    return components

# Test with disconnected graph
graph = {
    'A': ['B'],
    'B': ['A', 'C'],
    'C': ['B'],
    'D': ['E'],
    'E': ['D'],
    'F': []  # Isolated vertex
}

components = find_connected_components(graph)
print(f"Number of components: {len(components)}")
for i, comp in enumerate(components):
    print(f"  Component {i + 1}: {comp}")

***Figure 9.13:** DFS from each unvisited vertex discovers a new connected component.*

**Listing 9.14 — Union-Find for Components**

In [ ]:
class UnionFind:
    """
    Union-Find (Disjoint Set Union) data structure.
    Efficient for dynamic connectivity queries.
    """
    
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
        self.count = n  # Number of components
    
    def find(self, x):
        """Find root with path compression."""
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        """Union by rank."""
        px, py = self.find(x), self.find(y)
        if px == py:
            return False  # Already connected
        
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        
        self.count -= 1
        return True
    
    def connected(self, x, y):
        return self.find(x) == self.find(y)

# Test
edges = [(0, 1), (1, 2), (3, 4)]
uf = UnionFind(5)

print("Processing edges:")
for u, v in edges:
    uf.union(u, v)
    print(f"  Union({u}, {v}): {uf.count} components")

print(f"\n0 connected to 2: {uf.connected(0, 2)}")
print(f"0 connected to 3: {uf.connected(0, 3)}")

***Figure 9.14:** Union-Find provides near O(1) operations for connectivity queries.*

---
## 7. Cycle Detection


**Listing 9.15 — Cycle Detection in Undirected Graph**

In [ ]:
def has_cycle_undirected(graph):
    """
    Detect cycle in undirected graph using DFS.
    A cycle exists if we visit a vertex that's already visited
    and it's not the parent we came from.
    """
    visited = set()
    
    def dfs(vertex, parent):
        visited.add(vertex)
        
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                if dfs(neighbor, vertex):
                    return True
            elif neighbor != parent:
                return True  # Found cycle
        
        return False
    
    # Check all components
    for vertex in graph:
        if vertex not in visited:
            if dfs(vertex, None):
                return True
    
    return False

# Test
graph_with_cycle = {
    'A': ['B', 'C'],
    'B': ['A', 'C'],  # Cycle: A-B-C-A
    'C': ['A', 'B']
}

graph_no_cycle = {
    'A': ['B', 'C'],
    'B': ['A'],
    'C': ['A']
}

print(f"Graph with cycle: {has_cycle_undirected(graph_with_cycle)}")
print(f"Graph without cycle: {has_cycle_undirected(graph_no_cycle)}")

***Figure 9.15:** In undirected graphs, track parent to distinguish back edges from the edge we came from.*

**Listing 9.16 — Cycle Detection in Directed Graph**

In [ ]:
def has_cycle_directed(graph):
    """
    Detect cycle in directed graph using DFS coloring.
    White (0): unvisited
    Gray (1): in current path (recursion stack)
    Black (2): completely processed
    """
    WHITE, GRAY, BLACK = 0, 1, 2
    color = {v: WHITE for v in graph}
    
    def dfs(vertex):
        color[vertex] = GRAY
        
        for neighbor in graph[vertex]:
            if color[neighbor] == GRAY:
                return True  # Back edge = cycle
            if color[neighbor] == WHITE:
                if dfs(neighbor):
                    return True
        
        color[vertex] = BLACK
        return False
    
    for vertex in graph:
        if color[vertex] == WHITE:
            if dfs(vertex):
                return True
    
    return False

# Test
directed_with_cycle = {
    'A': ['B'],
    'B': ['C'],
    'C': ['A']  # Cycle: A -> B -> C -> A
}

directed_no_cycle = {
    'A': ['B', 'C'],
    'B': ['C'],
    'C': []
}

print(f"Directed with cycle: {has_cycle_directed(directed_with_cycle)}")
print(f"Directed without cycle (DAG): {has_cycle_directed(directed_no_cycle)}")

***Figure 9.16:** In directed graphs, a cycle exists if we find a back edge to a gray (in-progress) vertex.*

---
## 8. Topological Sort


> 📖 **Definition:** A **topological sort** of a DAG (Directed Acyclic Graph) is a linear ordering of vertices such that for every directed edge u→v, vertex u comes before v in the ordering. Used for task scheduling, build systems, course prerequisites.

**Listing 9.17 — Topological Sort (DFS)**

In [ ]:
def topological_sort_dfs(graph):
    """
    Topological sort using DFS.
    Add vertex to result after all descendants processed.
    """
    visited = set()
    result = []
    
    def dfs(vertex):
        visited.add(vertex)
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                dfs(neighbor)
        result.append(vertex)  # Add after processing descendants
    
    for vertex in graph:
        if vertex not in visited:
            dfs(vertex)
    
    return result[::-1]  # Reverse for topological order

# Course prerequisites
courses = {
    'Calculus I': [],
    'Calculus II': ['Calculus I'],
    'Linear Algebra': ['Calculus I'],
    'Differential Equations': ['Calculus II', 'Linear Algebra'],
    'Statistics': ['Calculus I'],
}

# Convert to adjacency list (prerequisite -> course)
graph = {course: [] for course in courses}
for course, prereqs in courses.items():
    for prereq in prereqs:
        graph[prereq].append(course)

order = topological_sort_dfs(graph)
print("Course order (take courses in this sequence):")
for i, course in enumerate(order, 1):
    print(f"  {i}. {course}")

***Figure 9.17:** DFS-based topological sort adds vertices to result in post-order, then reverses.*

**Listing 9.18 — Topological Sort (Kahn's Algorithm)**

In [ ]:
from collections import deque

def topological_sort_kahn(graph):
    """
    Kahn's algorithm: BFS-based topological sort.
    Start with vertices having no incoming edges.
    """
    # Calculate in-degrees
    in_degree = {v: 0 for v in graph}
    for v in graph:
        for neighbor in graph[v]:
            in_degree[neighbor] += 1
    
    # Start with zero in-degree vertices
    queue = deque([v for v in graph if in_degree[v] == 0])
    result = []
    
    while queue:
        vertex = queue.popleft()
        result.append(vertex)
        
        for neighbor in graph[vertex]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # Check for cycle
    if len(result) != len(graph):
        return None  # Cycle detected
    
    return result

# Build order example
tasks = {
    'design': ['code'],
    'code': ['test', 'docs'],
    'test': ['deploy'],
    'docs': ['deploy'],
    'deploy': []
}

order = topological_sort_kahn(tasks)
print("Build order:")
for i, task in enumerate(order, 1):
    print(f"  {i}. {task}")

***Figure 9.18:** Kahn's algorithm processes vertices with zero in-degree, detecting cycles if not all vertices are processed.*

---
## 9. Graph Applications


### Application 1: Bipartite Check

**Listing 9.19 — Bipartite Graph Check**

In [ ]:
from collections import deque

def is_bipartite(graph):
    """
    Check if graph is bipartite (2-colorable).
    A graph is bipartite if vertices can be divided into two sets
    such that no edge connects vertices in the same set.
    """
    color = {}
    
    for start in graph:
        if start in color:
            continue
        
        queue = deque([start])
        color[start] = 0
        
        while queue:
            vertex = queue.popleft()
            for neighbor in graph[vertex]:
                if neighbor not in color:
                    color[neighbor] = 1 - color[vertex]
                    queue.append(neighbor)
                elif color[neighbor] == color[vertex]:
                    return False  # Same color = not bipartite
    
    return True

# Test
bipartite_graph = {
    1: [2, 4],
    2: [1, 3],
    3: [2, 4],
    4: [1, 3]
}  # Square: can be 2-colored

not_bipartite = {
    1: [2, 3],
    2: [1, 3],
    3: [1, 2]
}  # Triangle: cannot be 2-colored

print(f"Square graph is bipartite: {is_bipartite(bipartite_graph)}")
print(f"Triangle graph is bipartite: {is_bipartite(not_bipartite)}")

***Figure 9.19:** Bipartite check uses BFS coloring. If neighbors have same color, not bipartite.*

### Application 2: Clone Graph

**Listing 9.20 — Deep Copy of Graph**

In [ ]:
class Node:
    def __init__(self, val, neighbors=None):
        self.val = val
        self.neighbors = neighbors if neighbors else []
    
    def __repr__(self):
        return f"Node({self.val})"

def clone_graph(node):
    """
    Deep copy a graph using DFS and hash map.
    Map original nodes to clones to handle cycles.
    """
    if not node:
        return None
    
    cloned = {}  # original -> clone
    
    def dfs(orig):
        if orig in cloned:
            return cloned[orig]
        
        copy = Node(orig.val)
        cloned[orig] = copy
        
        for neighbor in orig.neighbors:
            copy.neighbors.append(dfs(neighbor))
        
        return copy
    
    return dfs(node)

# Create a small graph: 1 -- 2
#                       |    |
#                       4 -- 3
n1, n2, n3, n4 = Node(1), Node(2), Node(3), Node(4)
n1.neighbors = [n2, n4]
n2.neighbors = [n1, n3]
n3.neighbors = [n2, n4]
n4.neighbors = [n1, n3]

clone = clone_graph(n1)
print(f"Original node 1: {n1}, neighbors: {n1.neighbors}")
print(f"Cloned node 1: {clone}, neighbors: {clone.neighbors}")
print(f"Same object? {n1 is clone}")
print(f"Same values? {n1.val == clone.val}")

***Figure 9.20:** Graph cloning uses a hash map to track cloned nodes and handle cycles.*

### Application 3: Word Ladder

**Listing 9.21 — Word Ladder (BFS)**

In [ ]:
from collections import deque

def word_ladder(begin_word, end_word, word_list):
    """
    Find shortest transformation sequence from begin to end.
    Each step changes one letter, intermediate words must be in list.
    """
    word_set = set(word_list)
    if end_word not in word_set:
        return 0
    
    queue = deque([(begin_word, 1)])
    visited = {begin_word}
    
    while queue:
        word, length = queue.popleft()
        
        if word == end_word:
            return length
        
        # Try changing each character
        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                
                if new_word in word_set and new_word not in visited:
                    visited.add(new_word)
                    queue.append((new_word, length + 1))
    
    return 0  # No transformation found

# Test
begin = "hit"
end = "cog"
words = ["hot", "dot", "dog", "lot", "log", "cog"]

result = word_ladder(begin, end, words)
print(f"Transform '{begin}' to '{end}'")
print(f"Word list: {words}")
print(f"Shortest path length: {result}")
print("Path: hit -> hot -> dot -> dog -> cog")

***Figure 9.21:** Word ladder is BFS on implicit graph where edges connect words differing by one letter.*

### Application 4: Number of Islands

**Listing 9.22 — Number of Islands (Grid DFS)**

In [ ]:
def num_islands(grid):
    """
    Count islands (connected components of 1s) in a grid.
    Grid is a 2D graph where adjacent cells are neighbors.
    """
    if not grid:
        return 0
    
    rows, cols = len(grid), len(grid[0])
    count = 0
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols:
            return
        if grid[r][c] != '1':
            return
        
        grid[r][c] = '#'  # Mark visited
        
        # Explore 4 directions
        dfs(r + 1, c)
        dfs(r - 1, c)
        dfs(r, c + 1)
        dfs(r, c - 1)
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                count += 1
                dfs(r, c)
    
    return count

# Test
grid = [
    ['1', '1', '0', '0', '0'],
    ['1', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0'],
    ['0', '0', '0', '1', '1']
]

print("Grid:")
for row in grid:
    print('  ' + ' '.join(row))

# Make a copy for display
grid_copy = [row[:] for row in grid]
result = num_islands([row[:] for row in grid])
print(f"\nNumber of islands: {result}")

***Figure 9.22:** Grid problems are graph problems! Each cell is a vertex, adjacent cells are edges.*

---
## 10. Common Patterns


### Pattern 1: Multi-Source BFS

**Listing 9.23 — Rotting Oranges**

In [ ]:
from collections import deque

def oranges_rotting(grid):
    """
    Multi-source BFS: start from all rotten oranges simultaneously.
    Return minutes until all oranges rot, or -1 if impossible.
    """
    rows, cols = len(grid), len(grid[0])
    queue = deque()
    fresh = 0
    
    # Find all rotten oranges and count fresh
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 2:
                queue.append((r, c, 0))  # (row, col, time)
            elif grid[r][c] == 1:
                fresh += 1
    
    if fresh == 0:
        return 0
    
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    max_time = 0
    
    while queue:
        r, c, time = queue.popleft()
        
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 1:
                grid[nr][nc] = 2
                fresh -= 1
                max_time = time + 1
                queue.append((nr, nc, time + 1))
    
    return max_time if fresh == 0 else -1

# Test
grid = [
    [2, 1, 1],
    [1, 1, 0],
    [0, 1, 1]
]
print(f"Minutes to rot all oranges: {oranges_rotting(grid)}")

***Figure 9.23:** Multi-source BFS starts from multiple sources simultaneously, useful for "spreading" problems.*

### Pattern 2: Graph from Matrix

**Listing 9.24 — Surrounded Regions**

In [ ]:
def solve(board):
    """
    Capture surrounded regions: flip 'O' to 'X' if surrounded.
    Key insight: 'O's connected to border cannot be captured.
    """
    if not board:
        return
    
    rows, cols = len(board), len(board[0])
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols:
            return
        if board[r][c] != 'O':
            return
        
        board[r][c] = 'S'  # Mark as safe
        dfs(r + 1, c)
        dfs(r - 1, c)
        dfs(r, c + 1)
        dfs(r, c - 1)
    
    # Mark 'O's connected to border as safe
    for r in range(rows):
        dfs(r, 0)
        dfs(r, cols - 1)
    for c in range(cols):
        dfs(0, c)
        dfs(rows - 1, c)
    
    # Flip: 'O' -> 'X' (surrounded), 'S' -> 'O' (safe)
    for r in range(rows):
        for c in range(cols):
            if board[r][c] == 'O':
                board[r][c] = 'X'
            elif board[r][c] == 'S':
                board[r][c] = 'O'

# Test
board = [
    ['X', 'X', 'X', 'X'],
    ['X', 'O', 'O', 'X'],
    ['X', 'X', 'O', 'X'],
    ['X', 'O', 'X', 'X']
]

print("Before:")
for row in board:
    print('  ' + ' '.join(row))

solve(board)
print("\nAfter capturing surrounded regions:")
for row in board:
    print('  ' + ' '.join(row))

***Figure 9.24:** Sometimes it's easier to find what NOT to capture (border-connected) than what to capture.*

### Pattern 3: Shortest Path in Weighted Graph

**Listing 9.25 — Dijkstra's Algorithm**

In [ ]:
import heapq

def dijkstra(graph, start):
    """
    Find shortest paths from start to all vertices.
    Time: O((V + E) log V) with heap.
    """
    distances = {v: float('inf') for v in graph}
    distances[start] = 0
    pq = [(0, start)]  # (distance, vertex)
    
    while pq:
        dist, vertex = heapq.heappop(pq)
        
        if dist > distances[vertex]:
            continue
        
        for neighbor, weight in graph[vertex]:
            new_dist = dist + weight
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                heapq.heappush(pq, (new_dist, neighbor))
    
    return distances

# Test
graph = {
    'A': [('B', 4), ('C', 2)],
    'B': [('A', 4), ('C', 1), ('D', 5)],
    'C': [('A', 2), ('B', 1), ('D', 8)],
    'D': [('B', 5), ('C', 8)]
}

distances = dijkstra(graph, 'A')
print("Shortest distances from A:")
for vertex, dist in sorted(distances.items()):
    print(f"  A -> {vertex}: {dist}")

***Figure 9.25:** Dijkstra's algorithm uses a priority queue to always process the nearest unvisited vertex.*

### Pattern 4: Course Schedule

**Listing 9.26 — Course Schedule (Cycle Detection)**

In [ ]:
from collections import defaultdict, deque

def can_finish(num_courses, prerequisites):
    """
    Check if all courses can be finished (no circular dependencies).
    Uses Kahn's algorithm for cycle detection.
    """
    graph = defaultdict(list)
    in_degree = [0] * num_courses
    
    for course, prereq in prerequisites:
        graph[prereq].append(course)
        in_degree[course] += 1
    
    queue = deque([i for i in range(num_courses) if in_degree[i] == 0])
    completed = 0
    
    while queue:
        course = queue.popleft()
        completed += 1
        
        for next_course in graph[course]:
            in_degree[next_course] -= 1
            if in_degree[next_course] == 0:
                queue.append(next_course)
    
    return completed == num_courses

# Test
print("Can finish 2 courses with [[1,0]]:", can_finish(2, [[1, 0]]))
print("Can finish 2 courses with [[1,0],[0,1]]:", can_finish(2, [[1, 0], [0, 1]]))

***Figure 9.26:** Course schedule is essentially cycle detection in a directed graph of prerequisites.*

### Pattern 5: Shortest Path with Constraints

**Listing 9.27 — Shortest Path with K Stops**

In [ ]:
from collections import defaultdict
import heapq

def find_cheapest_price(n, flights, src, dst, k):
    """
    Find cheapest flight from src to dst with at most k stops.
    Modified Dijkstra tracking (cost, stops, node).
    """
    graph = defaultdict(list)
    for u, v, price in flights:
        graph[u].append((v, price))
    
    # (cost, stops, node)
    pq = [(0, 0, src)]
    # Best cost to reach node with given stops
    best = {}
    
    while pq:
        cost, stops, node = heapq.heappop(pq)
        
        if node == dst:
            return cost
        
        if stops > k:
            continue
        
        # Skip if we've seen better path with fewer/equal stops
        if (node, stops) in best and best[(node, stops)] <= cost:
            continue
        best[(node, stops)] = cost
        
        for neighbor, price in graph[node]:
            heapq.heappush(pq, (cost + price, stops + 1, neighbor))
    
    return -1

# Test
flights = [[0,1,100],[1,2,100],[0,2,500]]
print(f"Cheapest 0->2 with 1 stop: {find_cheapest_price(3, flights, 0, 2, 1)}")
print(f"Cheapest 0->2 with 0 stops: {find_cheapest_price(3, flights, 0, 2, 0)}")

***Figure 9.27:** Modified Dijkstra tracks additional state (stops) to handle constraints.*

### Pattern 6: Graph Coloring

**Listing 9.28 — Possible Bipartition**

In [ ]:
from collections import defaultdict, deque

def possible_bipartition(n, dislikes):
    """
    Can we split n people into two groups where no two
    people in the same group dislike each other?
    This is bipartite checking!
    """
    graph = defaultdict(list)
    for a, b in dislikes:
        graph[a].append(b)
        graph[b].append(a)
    
    color = {}
    
    for person in range(1, n + 1):
        if person in color:
            continue
        
        queue = deque([person])
        color[person] = 0
        
        while queue:
            curr = queue.popleft()
            for enemy in graph[curr]:
                if enemy not in color:
                    color[enemy] = 1 - color[curr]
                    queue.append(enemy)
                elif color[enemy] == color[curr]:
                    return False
    
    return True

# Test
print(f"4 people, dislikes [[1,2],[1,3],[2,4]]: {possible_bipartition(4, [[1,2],[1,3],[2,4]])}")
print(f"3 people, dislikes [[1,2],[1,3],[2,3]]: {possible_bipartition(3, [[1,2],[1,3],[2,3]])}")

***Figure 9.28:** Many grouping problems reduce to bipartite checking (2-coloring).*

- **Graph** = vertices + edges; can be directed/undirected, weighted/unweighted
- **Adjacency list:** O(V + E) space, best for sparse graphs
- **BFS:** Level-order, shortest path (unweighted), uses queue
- **DFS:** Goes deep first, cycle detection, uses stack/recursion
- **Topological sort:** Order tasks with dependencies (DAG only)
- **Grid problems** are graph problems with implicit edges

---
## 11. Common Pitfalls


### Pitfall 1: Forgetting to Mark Visited

**Listing 9.29 — Visited Set Timing**

In [ ]:
from collections import deque

# WRONG: Mark visited when processing
def bfs_wrong(graph, start):
    visited = set()
    queue = deque([start])
    result = []
    
    while queue:
        vertex = queue.popleft()
        if vertex in visited:  # Check here
            continue
        visited.add(vertex)  # Mark here
        result.append(vertex)
        
        for neighbor in graph[vertex]:
            queue.append(neighbor)  # May add duplicates!
    
    return result

# RIGHT: Mark visited when adding to queue
def bfs_right(graph, start):
    visited = {start}  # Mark immediately
    queue = deque([start])
    result = []
    
    while queue:
        vertex = queue.popleft()
        result.append(vertex)
        
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                visited.add(neighbor)  # Mark before adding
                queue.append(neighbor)
    
    return result

graph = {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']}
print(f"Wrong approach processes more: queue operations")
print(f"Right approach: O(V + E) operations")

***Figure 9.29:** Mark visited when adding to queue, not when processing, to avoid duplicate work.*

### Pitfall 2: Wrong Cycle Detection in Undirected

**Listing 9.30 — Parent Tracking**

In [ ]:
# WRONG: Simple visited check (always finds "cycle")
def has_cycle_wrong(graph, start, visited=None):
    if visited is None:
        visited = set()
    
    visited.add(start)
    for neighbor in graph[start]:
        if neighbor in visited:
            return True  # Always true for undirected!
        if has_cycle_wrong(graph, neighbor, visited):
            return True
    return False

# RIGHT: Track parent to ignore edge we came from
def has_cycle_right(graph, start, visited=None, parent=None):
    if visited is None:
        visited = set()
    
    visited.add(start)
    for neighbor in graph[start]:
        if neighbor not in visited:
            if has_cycle_right(graph, neighbor, visited, start):
                return True
        elif neighbor != parent:  # Found visited that's not parent
            return True
    return False

# Test on simple path (no cycle)
path = {'A': ['B'], 'B': ['A', 'C'], 'C': ['B']}
print(f"Path A-B-C has cycle (wrong): {has_cycle_wrong(path, 'A')}")
print(f"Path A-B-C has cycle (right): {has_cycle_right(path, 'A')}")

***Figure 9.30:** In undirected graphs, the edge we came from doesn't count as a cycle.*

### Pitfall 3: Modifying Graph During Traversal

**Listing 9.31 — Safe Grid Modification**

In [ ]:
# Grid DFS with in-place marking (common pattern)
def flood_fill(grid, r, c, new_color):
    """Safe: mark before recursing."""
    rows, cols = len(grid), len(grid[0])
    old_color = grid[r][c]
    
    if old_color == new_color:
        return grid
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols:
            return
        if grid[r][c] != old_color:
            return
        
        grid[r][c] = new_color  # Mark BEFORE recursing
        dfs(r + 1, c)
        dfs(r - 1, c)
        dfs(r, c + 1)
        dfs(r, c - 1)
    
    dfs(r, c)
    return grid

# Test
grid = [
    [1, 1, 1],
    [1, 1, 0],
    [1, 0, 1]
]
result = flood_fill(grid, 1, 1, 2)
print("Flood fill result:")
for row in result:
    print(f"  {row}")

***Figure 9.31:** In grid DFS, modify the cell before recursing to prevent infinite loops.*

### Pitfall 4: BFS on Weighted Graphs

**Listing 9.32 — BFS vs Dijkstra**

In [ ]:
from collections import deque
import heapq

# WRONG: BFS for weighted graph
def bfs_shortest_weighted(graph, start, end):
    """BFS doesn't work for weighted graphs!"""
    visited = {start}
    queue = deque([(start, 0)])
    
    while queue:
        vertex, dist = queue.popleft()
        if vertex == end:
            return dist
        for neighbor, weight in graph[vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, dist + weight))
    return -1

# RIGHT: Dijkstra for weighted graph
def dijkstra_shortest(graph, start, end):
    distances = {v: float('inf') for v in graph}
    distances[start] = 0
    pq = [(0, start)]
    
    while pq:
        dist, vertex = heapq.heappop(pq)
        if vertex == end:
            return dist
        if dist > distances[vertex]:
            continue
        for neighbor, weight in graph[vertex]:
            new_dist = dist + weight
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                heapq.heappush(pq, (new_dist, neighbor))
    return -1

# Example where BFS fails
graph = {
    'A': [('B', 1), ('C', 5)],
    'B': [('C', 1)],
    'C': []
}
# Shortest A->C is A->B->C = 2, not A->C = 5

print(f"BFS result A->C: {bfs_shortest_weighted(graph, 'A', 'C')}")  # May give 5
print(f"Dijkstra result A->C: {dijkstra_shortest(graph, 'A', 'C')}")  # Gives 2

***Figure 9.32:** BFS only finds shortest path in unweighted graphs. Use Dijkstra for weighted graphs.*

### Pitfall 5: Disconnected Graphs

**Listing 9.33 — Handling Disconnected Graphs**

In [ ]:
# WRONG: Only traverses from one start vertex
def visit_all_wrong(graph, start):
    visited = set()
    def dfs(v):
        visited.add(v)
        for neighbor in graph[v]:
            if neighbor not in visited:
                dfs(neighbor)
    dfs(start)
    return visited

# RIGHT: Start DFS from each unvisited vertex
def visit_all_right(graph):
    visited = set()
    def dfs(v):
        visited.add(v)
        for neighbor in graph[v]:
            if neighbor not in visited:
                dfs(neighbor)
    
    for vertex in graph:
        if vertex not in visited:
            dfs(vertex)
    
    return visited

# Test with disconnected graph
disconnected = {
    'A': ['B'],
    'B': ['A'],
    'C': ['D'],  # Separate component
    'D': ['C']
}

print(f"Starting from A only: {visit_all_wrong(disconnected, 'A')}")
print(f"Visiting all components: {visit_all_right(disconnected)}")

***Figure 9.33:** Always iterate through all vertices to handle disconnected graphs.*

---
# 📝 Exercises


### Exercise 1: Valid Path  (⭐ Easy)

Given n vertices (0 to n-1) and edges, determine if there's a valid path from source to destination.

**Expected:** (Expected: Check reachability using BFS/DFS)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Build Graph:**

                        Create adjacency list from edges. Graph is undirected, so add both directions.
- **Hint 2 - BFS/DFS:**

                        Start from source, explore all reachable nodes using visited set.
- **Hint 3 - Check Destination:**

                        Return True if destination is visited during traversal.
</details>

In [ ]:
# ✏️ [EX1]
# Valid Path - Check reachability in graph

def valid_path(n, edges, source, destination):
    # Your code here
    pass

# Test your implementation (uncomment)
# n = 6
# edges = [[0,1],[0,2],[3,5],[5,4],[4,3]]
# print(valid_path(n, edges, 0, 5))  # False
# print(valid_path(n, edges, 0, 2))  # True

### Exercise 2: All Paths from Source to Target  (⭐⭐ Medium)

Given a DAG with n vertices (0 to n-1), find all paths from vertex 0 to vertex n-1.

**Expected:** (Expected: DFS with backtracking)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - DFS Start:**

                        Start DFS from node 0, track current path as you go.
- **Hint 2 - Target Check:**

                        When reaching node n-1, add a copy of current path to results.
- **Hint 3 - No Visited Set:**

                        DAG has no cycles, so no need for visited set. Just backtrack after exploring.
</details>

In [ ]:
# ✏️ [EX2]
# All Paths from Source to Target - DFS backtracking

def all_paths_source_target(graph):
    # Your code here
    pass

# Test your implementation (uncomment)
# graph = [[1,2],[3],[3],[]]  # 0->1,2; 1->3; 2->3; 3->none
# print(all_paths_source_target(graph))  # [[0,1,3],[0,2,3]]

### Exercise 3: Pacific Atlantic Water Flow  (⭐⭐⭐ Hard)

Given an m×n matrix of heights, find cells where water can flow to both Pacific (top/left) and Atlantic (bottom/right) oceans.

**Expected:** (Expected: DFS from ocean borders, find intersection)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Reverse Flow:**

                        Think in reverse: start from ocean borders, find cells that can reach the ocean.
- **Hint 2 - Two DFS:**

                        DFS from Pacific border (top + left), DFS from Atlantic border (bottom + right).
- **Hint 3 - Uphill in Reverse:**

                        Water flows downhill, so in reverse we go uphill: next cell height >= current.
</details>

In [ ]:
# ✏️ [EX3]
# Pacific Atlantic Water Flow - DFS from borders

def pacific_atlantic(heights):
    # Your code here
    pass

# Test your implementation (uncomment)
# heights = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
# print(pacific_atlantic(heights))

### Exercise 4: Course Schedule II  (⭐⭐⭐ Hard)

Return the order to take courses (topological sort), or empty array if impossible.

**Expected:** (Expected: Kahn's algorithm with order tracking)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Build Graph:**

                        Build adjacency list and compute in-degree for each course.
- **Hint 2 - Start with Zero In-Degree:**

                        Start with courses having no prerequisites (in-degree = 0).
- **Hint 3 - Detect Cycle:**

                        If not all courses processed, there's a cycle → return empty array.
</details>

In [ ]:
# ✏️ [EX4]
# Course Schedule II - Topological Sort

def find_order(num_courses, prerequisites):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(find_order(4, [[1,0],[2,0],[3,1],[3,2]]))  # [0,1,2,3] or [0,2,1,3]
# print(find_order(2, [[1,0],[0,1]]))  # [] (cycle)

### Exercise 5: Minimum Knight Moves  (⭐⭐⭐ Hard)

Find minimum moves for a knight to reach (x, y) from (0, 0) on an infinite chess board.

**Expected:** (Expected: BFS with knight L-shape moves)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - BFS:**

                        BFS from (0,0) gives shortest path. Each level = one move.
- **Hint 2 - Knight Moves:**

                        8 possible moves: (±1,±2) and (±2,±1).
- **Hint 3 - Symmetry:**

                        Use abs(x), abs(y) for symmetry. Limit search space to avoid infinite board.
</details>

In [ ]:
# ✏️ [EX5]
# Minimum Knight Moves - BFS on chess board

def min_knight_moves(x, y):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(min_knight_moves(2, 1))  # 1
# print(min_knight_moves(5, 5))  # 4

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Save this notebook** (Ctrl+S)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you SAVED the notebook first! (Ctrl+S)

import json, re, os, urllib.request

WEEK = "Week_09"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Find this notebook file ──
_nb_path = None

# VS Code
try:
    _nb_path = __vsc_ipynb_file__
except NameError:
    pass

# Colab
if not _nb_path:
    try:
        import google.colab
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if _candidates:
            _nb_path = _candidates[0]
    except ImportError:
        pass

# Fallback: search current dir
if not _nb_path:
    _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
    if len(_candidates) == 1:
        _nb_path = _candidates[0]

if not _nb_path or not os.path.exists(str(_nb_path)):
    print("⚠️  Could not auto-detect notebook file.")
    print("   Available .ipynb files:", [f for f in os.listdir(".") if f.endswith(".ipynb")])
    raise SystemExit("Please make sure the notebook is saved and in the current directory.")

print(f"📖 Reading {os.path.basename(str(_nb_path))}...")

with open(str(_nb_path), "r", encoding="utf-8") as _f:
    _nb = json.load(_f)

# ── Extract exercise answers ──
_answers = {}
for _cell in _nb["cells"]:
    if _cell["cell_type"] != "code":
        continue
    _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
    _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
    if _m:
        _ex_id = "ex" + _m.group(1)
        _lines = _src.split("\n")
        _clean = "\n".join(_lines[1:]).strip()
        _answers[_ex_id] = {
            "code": _clean,
            "modified": len(_clean) > 5
        }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure exercise cells still have the # ✏️ [EX...] tag.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
